<a href="https://colab.research.google.com/github/seoes/exhibition-visitor-tracker/blob/main/1-process-video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 설정

torch, yolo 설정 후 비디오 파일에 대하여 추적을 수행하여 txt파일을 생성하는 소스코드입니다.

## colab 사용시 실행

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

# 구글드라이브 사용시 현재 셀 실행하기
from google.colab import drive
drive.mount('/content/drive')

import os
HOME = os.getcwd()
%cd {HOME}

In [ ]:
!pip install gdown

!gdown --id 1vcKRPpv7ve1St-0rynhGa-H3lHMyCNt9 -O video/exhibition-64-sample.mp4
!gdown --id 1jEhErOcFKt7JrliQFzMar5yerHkTeu09 -O video/exhibition-64-full.mp4

# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1vcKRPpv7ve1St-0rynhGa-H3lHMyCNt9' -O exhibition-64-sample.mp4
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1ERyxsKGZmJZFgOBKGwVguQFqGJbYu575' -O exhibition-32-sample.mp4
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1Mcp8xRYhsaxE3SfGAUaeJk63rNNkx80d' -O exhibition-4-sample.mp4
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1jEhErOcFKt7JrliQFzMar5yerHkTeu09?' -O exhibition-64-full.mp4
# !wget --load-cookies /tmp/cookies.txt "https://drive.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1jEhErOcFKt7JrliQFzMar5yerHkTeu09' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1jEhErOcFKt7JrliQFzMar5yerHkTeu09" -O exhibition-64-full.mp4 && rm -rf /tmp/cookies.txt
# !wget --load-cookies /tmp/cookies.txt "https://drive.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1WJpbbx-w3QCFVmB0D0892vWVBCWdL3Kn' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1WJpbbx-w3QCFVmB0D0892vWVBCWdL3Kn" -O exhibition-64-test.mp4 && rm -rf /tmp/cookies.txt
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1QSneFB5hw85I9LMf14PHKh8P63kWRryc' -O exhibition-dark-sample.mp4
# !wget --no-check-certificate 'https://googledrive.com/host/exhibition/1jEhErOcFKt7JrliQFzMar5yerHkTeu09' -O exhibition-64-full.mp4


SAMPLE_VIDEO_64 = f"{HOME}/exhibition-64-sample.mp4"
SAMPLE_VIDEO_32 = f"{HOME}/exhibition-32-sample.mp4"
SAMPLE_VIDEO_4 = f"{HOME}/exhibition-4-sample.mp4"
FULL_VIDEO_64 = f"{HOME}/exhibition-64-full.mp4"
TEST_VIDEO_64 = f"{HOME}/exhibition-64-test.mp4"
SAMPLE_VIDEO_DARK = f"{HOME}/exhibition-dark-sample.mp4"

result_path = "/content/drive/MyDrive/result"
video_path = f"{HOME}/video"
filename = "exhibition-64-full.mp4"

## 로컬 환경에서 실행

In [9]:
import os
HOME = os.getcwd()
%cd {HOME}


filename = ""

with open(f"{HOME}/filename.txt", 'r') as file:
    for line in file:
        filename = line.strip()

result_path = f"{HOME}/result"
video_path = f"{HOME}/video"

/content


## 첫 설정

In [ ]:
import torch
import numpy as np
import cv2
%matplotlib inline
import matplotlib.pyplot as plt

from IPython.display import clear_output, display, Image
from collections import defaultdict

## YOLOv5 설치

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

## Supervision 설치

In [ ]:
!pip install supervision

import supervision as sv

colors = sv.ColorPalette.default()

VIDEO = video_path +"/" + filename

video_info = sv.VideoInfo.from_video_path(VIDEO)

# 2. 데이터 세팅

## Polygon 설정

In [ ]:
zone_coords = np.array([[378,260],[688,260],[1080,1745],[0,1745]])
polygon = zone_coords

## Annotator 설정

### ZoneAnnotator

Polygon 값에 따라 구역을 표시하는 Annotator

In [ ]:
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

zone_annotator = sv.PolygonZoneAnnotator(
        zone=zone,
        color=colors.by_idx(0),
        thickness=8,
        # text_thickness=8,
        # text_scale=4
    )

### BoundingBoxAnnotator

사람을 감싸는 직사각형 박스

In [ ]:
bounding_box_annotator = sv.BoundingBoxAnnotator(color=colors.by_idx(0),thickness=6)

### TraceAnnotator

사람의 동선을 따라가는 꼬리

In [ ]:
trace_annotator = sv.TraceAnnotator(color=colors.by_idx(0), trace_length=50,thickness=4)

### MaskAnnotator

사람을 덮는 Mask

In [ ]:
mask_annotator = sv.MaskAnnotator(color=colors.by_idx(0))

### LabelAnnotator

In [ ]:
label_annotator = sv.LabelAnnotator(color=colors.by_idx(0))

# 3. 비디오 분석

In [ ]:
tracker = sv.ByteTrack()

each_coord = []

def add_coordinates(detections, frame_number):
    global each_coord

    each_coord.append([])

    for detection in detections:

      coord = detection[0]

      conf = detection[2]

      center_x = int((coord[0] + coord[2]) / 2)
      bottom_y = int(coord[3])

      tracker_id = detection[4]

      each_coord[-1].append((frame_number, tracker_id, conf, center_x, bottom_y))

def process_frame(frame:np.ndarray, i) -> np.ndarray:
      print(f"Processing frame {i}...")

      results = model(frame,size=1280)
      detections = sv.Detections.from_yolov5(results)

      mask = zone.trigger(detections=detections)
      detections = detections[mask]

      detections = detections[(detections.class_id == 0) & (detections.confidence > 0.2)]
      detections = tracker.update_with_detections(detections)

      labels = [
          str(tracker_id)
          for tracker_id
          in detections.tracker_id
      ]

      frame = zone_annotator.annotate(scene=frame.copy())
      frame = bounding_box_annotator.annotate(scene=frame.copy(), detections=detections)
      frame = trace_annotator.annotate(scene=frame.copy(), detections=detections)
      frame = mask_annotator.annotate(scene=frame.copy(), detections=detections)
      frame = label_annotator.annotate(scene=frame.copy(), detections=detections, labels=labels)

      add_coordinates(detections, i)

      return frame


sv.process_video(source_path=VIDEO, target_path=f"{result_path}/{filename}",callback=process_frame)

for coords in each_coord:
  print(coords)

## 데이터 저장

#### 로컬에서 사용시 실행

#### colab에서 사용시 실행

In [ ]:
with open(f"{result_path}/{filename}-1-each_coord.txt"., 'w') as f:
    for coords in each_coord:
        for tuple_ in coords:
            line = ', '.join(map(str, tuple_))  # 튜플의 각 요소를 문자열로 변환하고 쉼표로 구분
            f.write(line + '\n')  # 파일에 한 줄씩 쓰기
        f.write('\n')  # 배열 각각을 구분하기 위해 빈 줄 추가